In [8]:
import sys
sys.path.append(r'C:/Users/ryoo/AppData/Local/gsas2main/GSAS-II')

import GSASII.GSASIIpath as GSASIIpath
GSASIIpath.binaryPath = r'C:/Users/ryoo/AppData/Local/gsas2main/GSAS-II/GSASII-bin/win_64_p3.13_n2.2'
GSASIIpath.SetBinaryPath(True)

import GSASII.GSASIIscriptable as G2sc
from GSASII import GSASIIspc as G2spc
from GSASII import GSASIIlattice as G2lat

import matplotlib.pyplot as plt 
from matplotlib.collections import LineCollection
from matplotlib.legend_handler import HandlerLineCollection
import numpy as np

In [12]:
def load_background_from_file(gpx, hist_index, background_file):
    """
    GSAS-II background parameter file (.pwdrbck)을 로드해서 histogram에 적용
    
    Parameters:
    -----------
    gpx : G2Project
        GSAS-II project object
    hist_index : int
        Histogram index
    background_file : str
        Path to .pwdrbck file
    """
    import ast
    
    hist = gpx.histogram(hist_index)
    
    with open(background_file, 'r') as f:
        lines = f.readlines()
    
    # Read background parameters 
    # ['chebyschev-1', True, 5, 13744.9516, -11410.0, 619.7199, 602.0776, 344.6025]
    bkg_line = lines[0].strip()
    if bkg_line.startswith('#'):
        bkg_line = lines[1].strip()
    
    bkg_params = ast.literal_eval(bkg_line)
    
    bkg_type = bkg_params[0]  # 'chebyschev-1'
    refine = bkg_params[1]     # True
    n_coeffs = bkg_params[2]   # 5
    coeffs = bkg_params[3:]    # [13744.9516, -11410.0, 619.7199, 602.0776, 344.6025]
    
    # background setting
    hist.set_refinements({
        "Background": {
            'type': bkg_type,
            'refine': refine,
            'no. coeffs': n_coeffs,
            'coeffs': coeffs
        }
    })
    
    gpx.save()
    print(f"Loaded background from {background_file}")
    print(f"Type: {bkg_type}, Coeffs: {n_coeffs}, Refine: {refine}")
    
    return bkg_params

In [36]:
"""
Create Pawley reflection from unit cell parameters from phase cif 
"""

def pawley_create(gpx, phase_name, dmin=None, dmax=None, neg_wt=0.0):
    ph  = gpx.phase(phase_name)
    gen = ph.data['General']
    SG  = gen['SGData']

    # d-range
    dmin = float(dmin if dmin is not None else gen.get('Pawley dmin', 1.0))
    dmax = float(dmax if dmax is not None else gen.get('Pawley dmax', 100.0))
    gen['doPawley']      = True
    gen['Pawley dmin']   = dmin
    gen['Pawley dmax']   = dmax
    gen['Pawley neg wt'] = float(neg_wt)

    # cell -> A6
    A6 = np.array(G2lat.cell2A(gen['Cell'][1:7]), float)
    HKL_mult_list = G2lat.GenHLaue(dmin, SG, A6)
    rd2 = getattr(G2lat, 'calc_rDsq', getattr(G2lat, 'Calc_rDsq', None))

    pawley_refs = []
    for h, k, l, mul in HKL_mult_list:
        d = 1.0/np.sqrt(rd2([h, k, l], A6))
        if dmin <= d <= dmax:
            # [h, k, l, mult, d, refine_flag, Fsq, sig(Fsq)]
            pawley_refs.append([
                float(h),
                float(k),
                float(l),
                int(mul),
                float(d),
                False,      # refine flag (나중에 따로 켬)
                100.0,      # 초기 Fsq는 0 말고 100 정도로
                10.0
            ])

    pawley_refs.sort(key=lambda x: x[4], reverse=True)
    ph.data['Pawley ref'] = pawley_refs

    # 여기서 phase 안의 histogram 설정을 해줘야
    # 이 phase가 그 histogram에서 Pawley로 실제 사용됨
    # 지금은 hist=0 하나만 쓴다고 했으니 그 이름만 넣자
    hist0 = gpx.histogram(0)
    hname = hist0.name
    ph_hist = ph.data['Histograms'].setdefault(hname, {})
    ph_hist['Use'] = True
    ph_hist['LeBail'] = True
    # Pawley만 돌릴 거면 scale은 고정
    if 'Scale' in ph_hist:
        ph_hist['Scale'][1] = False
    else:
        ph_hist['Scale'] = [1.0, False]

    gpx.save()
    return len(pawley_refs)


def _apply_refine_flags(ph, predicate, default=False):
    """predicate(h,k,l,mul,d,idx) → True 인 반사만 refine 체크"""
    refs = ph.data.get('Pawley ref', [])
    if not refs:
        print("No Pawley reflections found.")
        return 0
    n = 0
    for i, r in enumerate(refs):
        h, k, l, mul, d = int(r[0]), int(r[1]), int(r[2]), int(r[3]), float(r[4])
        flag = bool(predicate(h, k, l, mul, d, i))
        r[5] = flag if flag is not None else default
        n += int(r[5] is True)
    return n

def pawley_select_by_indices(gpx, phase_name, indices, on=True):
    idxset = set(indices)
    ph = gpx.phase(phase_name)
    _apply_refine_flags(ph, lambda h,k,l,mul,d,i: (i in idxset) == True and on, default=False)
    gpx.save()


def pawley_select_by_limit(gpx, phase_name, hist_index=0, wavelength=1.5406):
    """
    histogram의 Limits 범위 내에 있는 Pawley reflection만 refine하도록 설정
    
    Parameters:
    -----------
    gpx : G2Project
        GSAS-II project object
    phase_name : str
        Phase name
    hist_index : int
        Histogram index (default: 0)
    wavelength : float
        X-ray wavelength in Angstrom (default: 1.5406 for Cu K-alpha)
    
    Returns:
    --------
    int : Number of selected reflections
    """
    ph = gpx.phase(phase_name)
    hist = gpx.histogram(hist_index)
    limits = hist.data['Limits'][1]
    min_2theta, max_2theta = limits

    refs = ph.data.get('Pawley ref', [])
    if not refs:
        print("No Pawley reflections found.")
        return 0

    def d_to_2theta(d, lam):
        theta = np.arcsin(lam / (2*d))
        return 2*np.degrees(theta)

    n_selected = 0
    for r in refs:
        d = float(r[4])
        tt = d_to_2theta(d, wavelength)
        r[5] = (min_2theta <= tt <= max_2theta)
        n_selected += int(r[5])

    gpx.save()
    print(f"Selected {n_selected} reflections in [{min_2theta:.2f}, {max_2theta:.2f}]")
    return n_selected

In [ ]:
import numpy as np
# filename = r'C:/Users/ryoo/Desktop/LDRD/XRD/Beamline data/Baniecki_Mar 2025/HZO/HO_ZO_Graded_HT/HO_ZO_Graded_HT_0079.tif'
filename = r'C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Julydata\Combi_8_674V_5100us_presoak_200deg_60s_raster_sfpx63000um_scan003_processed_masked_pattern_00_1.xye'
project = r'C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\test.gpx'
calib = r'C:/Users/ryoo/Desktop/LDRD/XRD/Beamline data/Baniecki_Mar 2025/calibration/LaB6_detz200mm_th5deg_15s_03211554_0001.tif'
# instrument = r'C:/Users/ryoo/Desktop/LDRD/Code/XRD_processing/instrument_script.instprm'
instrument_july = r'C:/Users/ryoo/Desktop/LDRD/Code/XRD_processing/instrument_July.instprm'
controls = r'C:/Users/ryoo/Desktop/LDRD/Code/XRD_processing/image_control_oct.imctrl'
HfO2phase = r'C:/Users\ryoo\Desktop\LDRD\XRD\Open crystallography & Vesta\HfO2_orthorhombic_materialsproject_computed.cif'
ZrO2phase= r'C:\Users\ryoo\Desktop\LDRD\XRD\ICDD\4-ZrO2_P42nmc_Tetragonal\PDF Card - 04-005-4504.cif'
background_july = r'C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Background_July.pwdrbck'

# GPX_PATH = "practice.bak1.gpx"
# PHASES   = ["t-ZrO2", "o-HfO2"]

In [25]:
import sys
sys.path.append(r'C:/Users/ryoo/AppData/Local/gsas2main/GSAS-II')

import GSASII.GSASIIpath as GSASIIpath
GSASIIpath.binaryPath = r'C:/Users/ryoo/AppData/Local/gsas2main/GSAS-II/GSASII-bin/win_64_p3.13_n2.2'
GSASIIpath.SetBinaryPath(True)

import GSASII.GSASIIscriptable as G2sc
from GSASII import GSASIIspc as G2spc
from GSASII import GSASIIlattice as G2lat

import matplotlib.pyplot as plt 
from matplotlib.collections import LineCollection
from matplotlib.legend_handler import HandlerLineCollection
import numpy as np


In [ ]:
""" 
Create GPX file and import image or powder pattern 
"""
gpx = G2sc.G2Project(newgpx="C:/Users/ryoo/Desktop/LDRD/Code/XRD_processing/Test/test.gpx")  


# ## Import TIF image
# imlst = gpx.add_image(filename)

# ## Load controls that was used with LaB6 data
# imlst[0].loadControls(controls)

# ## Integrate 2D image 
# pwdrList = imlst[0].Integrate("test3")
# gpx.save()

# ## Import powder pattern
pwdlist = gpx.add_powder_histogram(filename, iparams = instrument_july)
gpx.save()

C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Julydata\Combi_8_674V_5100us_presoak_200deg_60s_raster_sfpx63000um_scan003_processed_masked_pattern_00_1.xye read by Reader Topas xye/qye or 2th Fit2D chi/qchi
Instrument parameters read: C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\instrument_July.instprm (G2 fmt) bank None
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx


In [38]:
## Load background from file
load_background_from_file(gpx,0, background_july)

## Set refinements
hist = gpx.histogram(0)
hist.set_refinements({"Limits": [27, 33],
                    "Background": {
                        "refine": False},
                      "Sample Parameters": ["Scale"],
                      })
gpx.save()

gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx
Loaded background from C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Background_July.pwdrbck
Type: chebyschev-1, Coeffs: 5, Refine: True
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx


In [39]:
"""
Add and adjust phase parameters : Refine unit cell, Create Pawley refinement parameters
"""

## Load project
gpx = G2sc.G2Project("C:/Users/ryoo/Desktop/LDRD/Code/XRD_processing/Test/test.gpx")

## add cif phase 
# gpx.add_phase(HfO2phase, phasename="o_HfO2")

phases = {
    "HfO2_pca21" : [HfO2phase, "P c a 21"],
    "ZrO2_p42nmc" : [ZrO2phase, "P 42 n m c"]
}

for phase in phases.items():
    phase_name = phase[0]
    directory = phase[1][0]
    spacegroup = phase[1][1]

    gpx.add_phase(directory, phasename=phase_name, histograms=gpx.histograms())

    # ph = gpx.phase(phase_name)  
    # gen = ph.data['General']

    # # ## adjusting phase
    err, SG = G2spc.SpcGroup(spacegroup)
    
    if err:
        raise ValueError(f"Bad space group '{spacegroup}': {err}")
    _ = G2spc.SGPrint(SG)        # 파생키(SGLaue, SGLatt, SGUniq, SpGrp 등) 보장

    ph = gpx.phase(phase_name)
    gen = ph.data['General']
    gen['SGData'] = SG

    # ## Refine unit cell
    gen['Cell'][0] = True

    ## Do Pawley refinement
    gen['doPawley'] = True
    gen['Pawley dmin'] = 2.5
    gen['Pawley dmax'] = 3.5
    gen['Pawley neg wt'] = 0.0
    gpx.save()

    ## Create Pawley
    n = pawley_create(gpx, phase_name)
    print("Pawley refs:", n)

    ## Select refine reflections
    # pawley_select_by_indices(gpx, phase_name, [0])

    pawley_select_by_limit(gpx, phase_name, hist_index=0, wavelength=1.54980)



    gpx.save()

C:\Users\ryoo\Desktop\LDRD\XRD\Open crystallography & Vesta\HfO2_orthorhombic_materialsproject_computed.cif read by Reader CIF
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx
Pawley refs: 4
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx
Selected 1 reflections in [27.00, 33.00]
gpx file saved as C:\Users\ryoo\Desktop\LDRD\Code\XRD_processing\Test\test.gpx
Symmetry element from input, -x,-y,z, not matched in GSAS-II setting
This CIF uses a space group setting not compatible with GSAS-II.

This is likely due to the space group being set in Origin 1 rather than 2.


CIF symops do not agree with GSAS-II, applying an origin 1->2 shift.

C:\Users\ryoo\Desktop\LDRD\XRD\ICDD\4-ZrO2_P42nmc_Tetragonal\PDF Card - 04-005-4504.cif read by Reader CIF
gpx file saved as C:\Users\ryoo\D